# Sentiment Analysis on Twitter Topics

In [1]:
import pandas as pd
from tqdm import tqdm

## 1. Importing Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
tt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/tweets_clean.csv')
tt

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,date,content,clean_text,length
0,2017-01-01,"Belief in #ClimateChange, #IntelligenceAgencie...","Belief ClimateChange, IntelligenceAgencies , E...",13.0
1,2017-01-01,Climate-Denying Trump's GOP Undermine The Cons...,Climate-Denying Trump's GOP Undermine The Cons...,18.0
2,2017-01-01,"'Instead of focusing on austerity alone, we se...","'Instead focusing austerity alone, see real ch...",14.0
3,2017-01-01,Totally agree. Are we looking to reduce carbon...,Totally agree. Are looking reduce carbon footp...,12.0
4,2017-01-01,@RachelNotley @shoffmanAB just want to make su...,RachelNotley shoffmanAB want make sure I freez...,12.0
...,...,...,...,...
347306,2020-01-31,Wetland biodiversity matters for life.\n2 Febr...,Wetland biodiversity matters life.\n2 February...,20.0
347307,2020-01-31,Please support @savewarrenfarm. Spread the wor...,Please support savewarrenfarm. Spread word don...,18.0
347308,2020-01-31,"150,000 Botanical and Animal Illustrations Ava...","150,000 Botanical Animal Illustrations Availab...",12.0
347309,2020-01-31,I thought it was all just a really bad dream.....,"I thought really bad dream... wars, al-Assad, ...",30.0


In [6]:
tt = tt.dropna()

In [7]:
tt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150660 entries, 0 to 347310
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        150660 non-null  object 
 1   content     150660 non-null  object 
 2   clean_text  150660 non-null  object 
 3   length      150660 non-null  float64
dtypes: float64(1), object(3)
memory usage: 5.7+ MB


## 2. Classifying Topic

In [8]:
import time
start_time = time.time()

!pip install bertopic

from bertopic import BERTopic
doc = [str(text) for text in tt.clean_text.tolist()]
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")
topics, probs = topic_model.fit_transform(doc)

end_time = time.time()
time_taken_seconds = end_time - start_time
time_taken_minutes = time_taken_seconds / 60
print(f"Time taken: {time_taken_minutes:.2f} minutes")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 82.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 83.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

Time taken: 8.05 minutes


In [9]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,76285,-1_warming_global_climate_amp
1,0,3862,0_canada_justintrudeau_tax_canadians
2,1,3274,1_footprint_carbon_flying_private
3,2,3220,2_sustainability_csr_business_esg
4,3,3088,3_australia_auspol_australian_morrison
...,...,...,...
868,867,10,867_mexico_seedsga_kaydos_biopasos
869,868,10,868_mfasco_kit_trauma_orange
870,869,10,869_gender_mainstreaming_women_empoweredwomen
871,870,10,870_guinguette_nantes_plantes_quai


In [10]:
# reduce number of topics
topic_model.reduce_topics(doc, nr_topics="auto")

In [11]:
# assign topic labels to every text
tt['topic'] = topics

# filter out the outliers
tt = tt[tt['topic'] != -1]

## 3. Analyzing Sentiment

### 3.1. Getting Sentiment Scores for Texts

In [12]:
# # Using SentimentIntensityAnalyzer

# import nltk
# nltk.download('vader_lexicon')
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# import pandas as pd

# # Instantiate the sentiment analyzer
# sia = SentimentIntensityAnalyzer()

# # Define a function to apply the sentiment analyzer to a given text
# def get_sentiment_scores(text):
#     return sia.polarity_scores(text)

In [13]:
# Using roberta
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# Define a function to apply the sentiment analyzer to a given text
def get_sentiment_scores(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # Define weights for negative, neutral, and positive sentiments
    weights = np.array([-1, 0, 1])

    # Compute the compound sentiment score
    compound_score = np.dot(scores, weights)
    
    return compound_score

In [15]:
start_time = time.time()

# Apply the sentiment analyzer to each text
tt["sentiment"] = tt["clean_text"].apply(get_sentiment_scores)

end_time = time.time()
time_taken_seconds = end_time - start_time
time_taken_minutes = time_taken_seconds / 60
print(f"Time taken: {time_taken_minutes:.2f} minutes")

Time taken: 253.45 minutes


In [16]:
tt

,date,content,clean_text,length,topic,sentiment
3,2017-01-01,Totally agree. Are we looking to reduce carbon...,Totally agree. Are looking reduce carbon footp...,12.0,1,0.417929
4,2017-01-01,@RachelNotley @shoffmanAB just want to make su...,RachelNotley shoffmanAB want make sure I freez...,12.0,1,-0.408179
5,2017-01-01,Mr B completed the @wwf_uk Carbon Footprint Ca...,Mr B completed wwf uk Carbon Footprint Calcula...,15.0,611,0.480435
6,2017-01-01,We have always offset our carbon footprint by ...,We always offset carbon footprint adopting end...,17.0,106,0.615385
8,2017-01-01,Global warming. Don´t worry- I know these plan...,"Global warming. Dont worry- I know plants, I m...",15.0,1,0.068823
...,...,...,...,...,...,...
347303,2020-01-31,"Biodiversity Heritage Library Makes 150,000 An...","Biodiversity Heritage Library Makes 150,000 An...",12.0,411,0.473652
347304,2020-01-31,"Biodiversity Heritage Library Makes 150,000 An...","Biodiversity Heritage Library Makes 150,000 An...",14.0,411,0.531726
347305,2020-01-31,"Panama, land bridge of the Americas and two lo...","Panama, land bridge Americas two long coastlin...",22.0,133,0.625658
347308,2020-01-31,"150,000 Botanical and Animal Illustrations Ava...","150,000 Botanical Animal Illustrations Availab...",12.0,411,0.375749


In [18]:
tt.to_csv('/content/drive/MyDrive/Colab Notebooks/data/tweets_sentiment.csv', index=False)

### 3.2. Getting Sentiment Scores for Topics

In [19]:
# Calculate the average sentiment per topic
topic_sen = tt.groupby('topic')['sentiment'].mean().round(2).reset_index()
topic_sen.columns = ['Topic', 'Avg_Sentiment']

# Merge the topic counts and average sentiment dataframes
topic_info_sen = pd.merge(topic_model.get_topic_info(), topic_sen, on='Topic')

# Display the resulting dataframe
topic_info_sen

,Topic,Count,Name,Avg_Sentiment
0,0,3862,0_canada_tax_justintrudeau_canadians,-0.25
1,1,3607,1_australia_auspol_australian_morrison,-0.19
2,2,3274,2_footprint_carbon_flying_private,0.47
3,3,3220,3_sustainability_business_csr_esg,-0.38
4,4,2671,4_trump_realdonaldtrump_gore_al,0.20
...,...,...,...,...
618,618,10,618_mexico_seedsga_kaydos_blackeyedpea,0.00
619,619,10,619_mfasco_kit_trauma_orange,-0.22
620,620,10,620_guinguette_nantes_plantes_quai,0.36
621,621,10,621_disruption_discontinuous_confrontational_c...,-0.14


In [20]:
topic_info_sen.head(30)

,Topic,Count,Name,Avg_Sentiment
0,0,3862,0_canada_tax_justintrudeau_canadians,-0.25
1,1,3607,1_australia_auspol_australian_morrison,-0.19
2,2,3274,2_footprint_carbon_flying_private,0.47
3,3,3220,3_sustainability_business_csr_esg,-0.38
4,4,2671,4_trump_realdonaldtrump_gore_al,0.20
5,5,2639,5_policy_environmental_students_class,-0.14
6,6,1727,6_food_meat_farmers_agriculture,0.06
7,7,1589,7_eu_brexit_labour_uk,-0.26
8,8,1348,8_la_de_les_le,0.16
9,9,1258,9_trees_tree_forests_planting,-0.19


In [21]:
topic_model.get_representative_docs(0)

["It's time Canada's Minister Environment Climate Change resign. It's clear government understand take real action address climate crisis. Please, good Canada, cathmckenna - resign. cdnpoli gpc ndp",
 'Back carbon pricing "My friends Ontario doesnt need carbon tax reduce greenhouse gas emissions. We released comprehensive environmental plan plan reach targets without carbon tax Were leading country emissions reductions.',
 'JustinTrudeau I need explain Liberal Party making life much affordable Canadians. You unachievable climate action plan Canadians paying HIGH PRICE. TrudeauMustGo cdnpoli elxn43']

In [22]:
topic_info_sen.to_csv('/content/drive/MyDrive/Colab Notebooks/data/tweets_topic_sentiment.csv', index=False)

In [23]:
# SentimentIntensityAnalyzer can not detect sarcasm
text = "Donald Trump better pray there is no global warming, because he's one easily melted precious snowflake.   ❄️❄️❄️❄️ https://t.co/p278S4hbVz"
get_sentiment_scores(text)

-0.768059141933918